In [1]:
!pip install google-generativeai tqdm python-dotenv aiohttp requests beautifulsoup4 huggingface_hub

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os, json, time, asyncio
from tqdm import tqdm
import google.generativeai as genai
from huggingface_hub import hf_hub_download
import requests
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

print("API Loaded Successfully!")

API Loaded Successfully!


In [3]:
def safe_parse_json(text: str):
    import re
    text = text.strip()
    try: return json.loads(text)
    except: pass
    m = re.search(r'(\{(?:.|\n)*?\})', text)
    if m:
        try: return json.loads(m.group(1))
        except: pass
    m = re.search(r'(\[(?:.|\n)*?\])', text)
    if m:
        return json.loads(m.group(1))
    return text

In [4]:
class GeminiLLM:
    def __init__(self, model='models/gemini-2.5-pro'):
        self.model = genai.GenerativeModel(model)

    def generate(self, prompt):
        try:
            resp = self.model.generate_content(prompt)
            return resp.text
        except Exception as e:
            return f'[ERROR: {e}]'

In [5]:
class BaseAgent:
    def __init__(self, name, llm):
        self.name = name
        self.llm = llm
    async def run_task(self, task): raise NotImplementedError

In [6]:
class WebResearcherAgent(BaseAgent):
    async def run_task(self, task):
        topic = task["instruction"]
        prompt = f"""
Create a well-formatted Markdown article explaining:

    {topic}

Formatting rules:
- Use ### for main headings
- Use #### for subheadings
- Use bullet points
- Use spacing
- No JSON, no code blocks
- Make it look like a nicely formatted blog post
- Include links to good sources

Example format:

### **Topic Title**

#### **Section 1**
* point 1
* point 2

#### **Section 2**
* explanation
* link: https://example.com

Now write the formatted output:
"""
        response = self.llm.generate(prompt)
        return {
            "id": task["id"],
            "status": "done",
            "result": response
        }



In [7]:
class Controller:
    def __init__(self):
        self.llm = GeminiLLM()
        self.workers = {
            'web_researcher': WebResearcherAgent('web_researcher', self.llm)
        }

    def plan(self, goal):
        return [{"id": "t1", "role": "web_researcher", "instruction": goal, "context": {}}]

    async def execute(self, tasks):
        results = []
        for t in tasks:
            agent = self.workers[t['role']]
            results.append(await agent.run_task(t))
        return results

In [8]:
from IPython.display import Markdown, display

controller = Controller()
goal = input("Enter your goal: ")

tasks = controller.plan(goal)
print("TASKS:\n", tasks)

results = await controller.execute(tasks)

print("\nRESULTS:\n")
for r in results:
    display(Markdown(r["result"]))


Enter your goal:  explain akbar ?


TASKS:
 [{'id': 't1', 'role': 'web_researcher', 'instruction': 'explain akbar ?', 'context': {}}]

RESULTS:



### **Akbar the Great: The Architect of the Mughal Empire**

Jalal-ud-din Muhammad Akbar (1542-1605), more commonly known as Akbar the Great, was the third Mughal emperor and is widely regarded as one of the most influential and successful rulers in Indian history. His reign transformed the Mughal domain into a vast, centralized, and multicultural empire that left a lasting impact on the Indian subcontinent.

***

#### **Early Life and Rise to Power**

*   Born to the Mughal emperor Humayun and Hamida Banu Begum, Akbar's early life was marked by instability as his father was in exile.
*   He ascended to the throne in 1556 at the young age of 13, following his father's sudden death.
*   His early reign was guided by his regent, Bairam Khan, a capable general who helped him secure the empire.
*   A pivotal moment was the Second Battle of Panipat in 1556, where Akbar's forces defeated the Hindu general Hemu, firmly establishing Mughal rule in Northern India.

***

#### **Military Expansion and Consolidation**

*   Akbar was a brilliant military strategist and commander who spent much of his reign expanding the Mughal Empire.
*   Through a series of successful military campaigns, he conquered vast territories, including Malwa, Gujarat, Bengal, Kashmir, and parts of the Deccan.
*   He consolidated his power by subduing rebellious Afghan chiefs and incorporating Rajput kingdoms into his empire through both warfare and strategic matrimonial alliances. This policy of diplomacy helped ensure the loyalty of the powerful Rajput warrior class.

***

#### **Administrative Genius**

Akbar's true legacy lies in his administrative reforms, which created a strong and efficient framework for governance.

*   **Mansabdari System:** He introduced a sophisticated system for ranking government officials, both military and civil. Each *mansabdar* (rank-holder) was responsible for maintaining a specific number of cavalrymen and was paid a salary. This system ensured a loyal and organized bureaucracy.
*   **Centralized Revenue System:** Under his finance minister, Raja Todar Mal, Akbar implemented a land revenue system called the *Zabt* system. Land was carefully measured, and taxes were fixed based on the fertility and yield of the soil, making the system more equitable and predictable for peasants.
*   **Provincial Administration:** He divided the empire into 12 provinces or *subas*, each governed by a *subadar* (governor) who was responsible for maintaining law and order.

***

#### **Policy of Religious Tolerance (Sulh-i-Kul)**

Akbar is most celebrated for his liberal religious policies and his promotion of harmony among different faiths.

*   He promoted the policy of *Sulh-i-Kul*, which translates to "universal peace" or "peace with all." This policy aimed to treat all subjects equally, regardless of their religion.
*   In 1564, he abolished the *jizya*, a poll tax levied on non-Muslims, a move that was widely welcomed by his Hindu subjects.
*   He established the *Ibadat Khana* (House of Worship) at his capital, Fatehpur Sikri, where he held debates with scholars from various religions, including Islam, Hinduism, Christianity, Jainism, and Zoroastrianism.
*   He even promulgated a new syncretic faith called **Din-i Ilahi** ("Religion of God"), which blended the best elements of different religions. While it never gained a mass following, it symbolized his vision of a united and tolerant empire.

***

#### **Patronage of Art and Culture**

Akbar's reign was a golden age for Mughal art, architecture, and literature.

*   He commissioned magnificent architectural projects, most notably the new capital city of **Fatehpur Sikri** and the grand Agra Fort.
*   Mughal painting flourished under his patronage, blending Persian styles with Indian traditions.
*   His court was home to the **Navaratnas** (Nine Jewels), a group of nine extraordinary individuals who excelled in their respective fields. This group included figures like the witty advisor Birbal, the legendary musician Tansen, and the historian Abul Fazl, who wrote the *Akbarnama*, a detailed chronicle of Akbar's reign.

***

#### **Further Reading**

To learn more about this remarkable ruler, explore these resources:

*   Encyclopedia Britannica: [https://www.britannica.com/biography/Akbar](https://www.britannica.com/biography/Akbar)
*   World History Encyclopedia: [https://www.worldhistory.org/Akbar/](https://www.worldhistory.org/Akbar/)
*   Wikipedia: [https://en.wikipedia.org/wiki/Akbar](https://en.wikipedia.org/wiki/Akbar)